### using all packages

In [1]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

### using module

In [2]:
include("HighDimMixedModels.jl")
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [3]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)
df

,y,a,b,c,d
,Float64,Int64,Float64,String,Float64
1,0.585195,1,0.236782,d,0.752223
2,0.0773379,2,0.943741,e,0.314815
3,0.716628,3,0.445671,f,0.858522
4,0.320357,4,0.763679,d,0.698713
5,0.653093,5,0.145071,e,0.757746
6,0.236639,6,0.021124,f,0.419294
7,0.709684,7,0.152545,d,0.412607
8,0.557787,8,0.617492,e,0.454589
9,0.05079,9,0.481531,f,0.380933


In [6]:
#MixedModels.datasets()
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
first(df,5)

,herd,period,incid,hsz
,String,Float64,Int8,Int8
1,H01,1.0,2,14
2,H01,2.0,3,12
3,H01,3.0,4,9
4,H01,4.0,0,5
5,H02,1.0,3,22


In [7]:
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [8]:
f = @formula(period ~ herd + incid + hsz)
contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

Dict{Symbol, UnionAll} with 3 entries:
  :hsz   => ContinuousTerm
  :herd  => CategoricalTerm
  :incid => ContinuousTerm

### Construct object

#### 1.Number of variables indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

#### 2.indices indicates M,X,Z

In [ ]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

#### 3.variable names indicates M,X,Z

In [9]:
#HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "incid", "hsz","herd")

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


In [10]:
first(HMM.M.M,5)

5-element Vector{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0

In [11]:
first(HMM.X.X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [19]:
size(HMM.Z.Z)

(56, 14)

In [ ]:
y, pred = modelcols(form, df);
terms = form.rhs.terms
M = highDimMat(modelmatrix(terms[idOfHDM],df))
X = XMat(modelmatrix(terms[idOfXMat],df))

#preTerms = Vector(1:length(form.rhs.terms))
#idOfReMat = preTerms[preTerms .∉ Ref(vcat(idOfHDM, idOfXmat))]
Z = ReMat(modelmatrix(terms[idOfReMat],df))

### Optimization

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$
To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [50]:
include("HighDimMixedModels.jl")
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "incid", "hsz","herd")

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


In [54]:
sigma, betaM, betaX, opt = HighDimMM.fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.5, 1.5]
negLog = -55.42022892537054
The initial object value is -55.42022892537054
sigma = [1.5, 1.5]
negLog = -55.42022892537054
OPTBL: starting point [1.5, 1.5]
sigma = [1.5, 1.5]
negLog = -55.42022892537054
sigma = [2.625, 1.5]
negLog = -56.410930728972986
sigma = [2.625, 2.625]
negLog = -55.703752290415146
sigma = [3.5406536917518467, 0.8463920771737735]
negLog = -67.55427132321276
sigma = [4.664039325095646, 0.7861449295504178]
negLog = -70.77586409759834
sigma = [5.468776021004419, 1.5722898591008356]
negLog = -58.566262269973535
sigma = [4.725648228604872, 0.2270290265317405]
negLog = -177.0870938482737
sigma = [4.210998764297321, 2.7755575615628914e-17]
negLog = Inf
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [NaN, NaN]
negLog = NaN
sigma = [4.724585329271449, 0.28300127417115684]
negLog = -145.10626768354598
sigma = [NaN, NaN]
negLog = NaN
sigma = [4.7722908084308795, 0.22777802921369125]
negLog = -176.5723495949748
sigma = [4.72519044885

In [55]:
HMM

Linear mixed model fit by REML
 period ~ herd + incid + hsz
 REML criterion at convergence: -4.72013520234647e6


σ_z : 4.746277173805859
σ : 8.824869914382332e-6
β_M : [-0.2355940802560706]
β_X : Float64[]


In [53]:
HMM.optsum

|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.5, 1.5]         |
| Initial objective value  | -55.42022892537054 |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_BOBYQA`        |
| Lower bounds             | [0.0, 0.0]         |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 1.0e-5             |
| `xtol_abs`               | [0.0, 0.0]         |
| `initial_step`           | [1.125, 1.125]     |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 22                 |
| Final parameter vector   | [1.3156, 0.1566]   |
| Final objective value    | -253.7661          |
| Return code              | `XTOL_REACHED`     |


In [45]:
opt

|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.0, 1.0]         |
| Initial objective value  | -61.76979106719719 |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_BOBYQA`        |
| Lower bounds             | [0.0, 0.0]         |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 1.0e-5             |
| `xtol_abs`               | [0.0, 0.0]         |
| `initial_step`           | [0.75, 0.75]       |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 22                 |
| Final parameter vector   | [0.9947, 0.054]    |
| Final objective value    | -743.9667          |
| Return code              | `XTOL_REACHED`     |


In [ ]:
display(HMM)

In [ ]:
opt

In [ ]:
show(opt)

In [ ]:
display(opt)

In [ ]:
sigma

After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [ ]:
beta

In [ ]:
data = MixedModels.dataset(:cbpp)

In [ ]:
using CSV

In [ ]:
CSV.write("cbpp",data)

In [ ]:
pwd

In [ ]:
isnothing(12)